In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd

In [2]:
builder = (
    SparkSession.builder.appName("DataWarehouse-ETL")
    # Memory configurations for ETL processing
    .config("spark.driver.memory", "4g")
    .config("spark.executor.memory", "4g")
    .config("spark.driver.maxResultSize", "2g")
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
    # Jars for Delta Lake, S3, and MySQL
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.4.1,com.amazonaws:aws-java-sdk-bundle:1.12.262," \
    "io.delta:delta-spark_2.13:4.0.0," \
    "com.mysql:mysql-connector-j:8.0.33")
    # Delta Lake
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    # MinIO (S3A) - Source
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9900")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    # S3A performance configs
    .config("spark.hadoop.fs.s3a.connection.timeout", "60000")
    .config("spark.hadoop.fs.s3a.connection.request.timeout", "60000")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.attempts.maximum", "3")
    .config("spark.hadoop.fs.s3a.retry.limit", "3")
)

spark = builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/10 11:26:34 WARN Utils: Your hostname, bnguyen-lenovo, resolves to a loopback address: 127.0.1.1; using 192.168.1.9 instead (on interface wlp2s0)
25/10/10 11:26:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/bnguyen/Desktop/finance_analytics/venv/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/bnguyen/.ivy2.5.2/cache
The jars for the packages stored in: /home/bnguyen/.ivy2.5.2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
io.delta#delta-spark_2.13 added as a dependency
com.mysql#mysql-connector-j added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f0d99f54-33be-4835-a7b8-6d9ae2755422;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.4.1 

In [3]:
# Check the test table where CDC streaming is writing
df_test = spark.read.format("delta").load("s3a://rootdb/users")
df_test.filter(df_test.client_id == 999999999999).show()


25/10/10 11:26:43 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
25/10/10 11:26:46 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------+-----------+--------------+----------+-----------+------+-------------+--------+---------+-----------------+-------------+----------+------------+----------------+
|   client_id|current_age|retirement_age|birth_year|birth_month|gender|      address|latitude|longitude|per_capita_income|yearly_income|total_debt|credit_score|num_credit_cards|
+------------+-----------+--------------+----------+-----------+------+-------------+--------+---------+-----------------+-------------+----------+------------+----------------+
|999999999999|         30|            65|      1993|          1|Other |Dummy Address|    NULL|     NULL|             NULL|         NULL|      NULL|         500|               0|
+------------+-----------+--------------+----------+-----------+------+-------------+--------+---------+-----------------+-------------+----------+------------+----------------+

